In [ ]:
import requests
import pandas as pd
from google.cloud import bigquery

# Function to fetch data from Yahoo Finance API
def get_yahoo_finance_data(symbol, start_date, end_date):
    base_url = "https://query1.finance.yahoo.com/v7/finance/download"
    params = {
        'symbol': symbol,
        'period1': str(int(pd.to_datetime(start_date).timestamp())),
        'period2': str(int(pd.to_datetime(end_date).timestamp())),
        'interval': '1d',
        'events': 'history',
    }

    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Assuming the response is a CSV file, you can read it into a DataFrame
        df = pd.read_csv(pd.compat.StringIO(response.text))
        return df
    else:
        print(f"Error fetching data from Yahoo Finance API. Status Code: {response.status_code}")
        return None

# Function to fetch data from World Bank Open Data API
def get_world_bank_data(indicator, country_code, start_date, end_date):
    base_url = "http://api.worldbank.org/v2/country/{0}/indicator/{1}".format(country_code, indicator)
    params = {
        'format': 'json',
        'date': f"{start_date}:{end_date}",
    }

    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract data from the response
        data = response.json()
        # Assuming the response structure, you can retrieve the relevant data
        df = pd.json_normalize(data[1])
        return df
    else:
        print(f"Error fetching data from World Bank API. Status Code: {response.status_code}")
        return None

# Function to fetch data from Google Cloud BigQuery
def get_bigquery_data(query):
    client = bigquery.Client()
    query_job = client.query(query)
    results = query_job.result()

    # Assuming the result is a pandas DataFrame
    df = results.to_dataframe()
    return df

# Example usage
yahoo_data = get_yahoo_finance_data('AAPL', '2022-01-01', '2022-12-31')
print("Yahoo Finance Data:")
print(yahoo_data.head())

world_bank_data = get_world_bank_data('NY.GDP.MKTP.CD', 'USA', '2022', '2022')
print("\nWorld Bank Data:")
print(world_bank_data.head())

# Example BigQuery SQL query to retrieve data
bigquery_query = "SELECT * FROM `bigquery-public-data.samples.wikipedia` LIMIT 5"
google_cloud_data = get_bigquery_data(bigquery_query)
print("\nGoogle Cloud Public Dataset - Wikipedia Sample Data:")
print(google_cloud_data.head())
